# Mini projet Qualité de Données : Détections des doublons
## ***Christophe COMPAIN / Sander COHEN***

### Objectif et Données Disponibles
L'objectif du projet est d'identifier les logiciels vendus sur les deux plateformes.

Pour ce faire, nous disposons des données pour chacune des plateformes isolément, respectivement dans les fichiers ***Company1.csv*** et ***Company2.csv***. 

### Import packages, Variables Globales et import csv

In [1]:
import pandas as pd
import nltk
import time
import numpy as np

In [2]:
path = "D:\\OneDrive - Université Paris-Dauphine\\Bureau\\Cours Master\\12-Qualité de Données\\\Projet\\mini-projet\\"
file1= "SampleData\\Sample_Company1.csv"
file2= "SampleData\\Sample_Company2.csv"
real= "SampleData\\Sample_Groud_truth_mappings.csv"

In [3]:
company1 = pd.read_csv(path+file1)
company2 = pd.read_csv(path+file2)
ground_truth_matches = pd.read_csv(path+real)

### Exploration des données

In [4]:
company1.head(5)

,id,title,description,manufacturer,price
0,b0000aka82,studyworks! teaching pro: middle school math &...,studyworks teaching pro middle school math & s...,global software publishing,29.99
1,b000bnb72g,movies on psp,x-zoom movies on psp is the first commercial v...,x-oom,0.00
2,b0002e3g6o,eastwest bsendorfer 290 grand piano virtual in...,the 5-star awarded pmi bdorfer 290 is simply t...,eastwest,199.95
3,b00008ajjc,passwords plus 1.0,passwords plus stores all your personal inform...,dataviz,29.99
4,b0006g2wke,microsoft licenses project svr sa govt (h2200301),microsoft project server win32 english softwar...,microsoft licenses,3601.40


In [5]:
company2.head(5)

,id,name,description,manufacturer,price
0,12244614697089679523,production prem cs3 mac upgrad,adobe cs3 production premium mac upgrade from ...,adobe software,805.99
1,5781318040297863663,ipswitch wd-1000-0900 ws_ftp pro 9 sdk w/ ws_f...,no description available,ipswitch,325.51
2,9949258255997064102,money prem 2007 cd minibox,money premium 2007 win32 english north america...,microsoft,63.99
3,9761533219806554318,adobe indesign cs3 for mac upgrade,system requirements powerpc g4 or g5 or intel ...,NaN,205.99
4,18226456193742595288,bamboo technology llc vault360 combo,capitalizing on the popularity of the ipod psp...,NaN,24.58


In [6]:
ground_truth_matches.head(5)

,idCompany1,idCompany2
0,b0002ibev4,1887899244694755891
1,b0007lw22g,18398718226932431716
2,b000ofnri8,12244614697089679523
3,b0007d8r5k,13775362651326388438
4,b00099qrok,9755705822363275907


#### Observation d'un premier duplicat

In [8]:
company1[company1.id == ground_truth_matches.idCompany1[1]]

,id,title,description,manufacturer,price
32,b0007lw22g,apple ilife '06 (mac dvd) [older version],ilife '06 is the easiest way to make the most ...,apple computer,79.0


In [9]:
company2[company2.id == ground_truth_matches.idCompany2[1]]

,id,name,description,manufacturer,price
46,18398718226932431716,ilife '06 mac - apple - ma166z/a,ilife '06 the must-have update to apple's awar...,NaN,47.8 gbp


In [16]:
##Methode Christophe
number_of_matches = 0
matches=[]
for i in range(len(company1)):
    tokens1name = nltk.word_tokenize(company1.iloc[i,1])
    ng1_tokensname = set(nltk.ngrams(tokens1name, n=1))
    #print(ng1_tokensname)
    for j in range(len(company2)):
        tokens2name = nltk.word_tokenize(company2.iloc[j,1])
        ng2_tokensname = set(nltk.ngrams(tokens2name, n=1))
        jd_ng1_ng2_name = nltk.jaccard_distance(ng1_tokensname, ng2_tokensname)
        name_score = nltk.edit_distance(company1.iloc[i,1], company2.iloc[j,1])
        if ((jd_ng1_ng2_name <= 0.75) or name_score<=1) :
            number_of_matches = number_of_matches +1
            matches.append((company1.iloc[i,0],company2.iloc[j,0]))
print("Number of matches: {}".format(number_of_matches))
matches_df = pd.DataFrame(matches)
matches_df.columns= ['idCompany1','idCompany2']
diff_df = pd.merge(ground_truth_matches, matches_df, how='outer', indicator='Exist')
true_positives = diff_df[diff_df.Exist=='both']
false_positives = diff_df[diff_df.Exist=='right_only']
false_negatives = diff_df[diff_df.Exist=='left_only']
print("Number of true positives: {}".format(len(true_positives)))
print("Number of false positives: {}".format(len(false_positives)))
print("Number of false negatives: {}".format(len(false_negatives)))
precision = len(true_positives)/(len(true_positives)+ len(false_positives))
print("Precision: {}".format(precision))
recall = len(true_positives)/(len(true_positives)+ len(false_negatives))
print("Recall: {}".format(recall))
f_measure = 2*(precision*recall)/(precision+recall)
print("F measure: {}".format(f_measure))

Number of matches: 85
Number of true positives: 70
Number of false positives: 15
Number of false negatives: 33
Precision: 0.8235294117647058
Recall: 0.6796116504854369
F measure: 0.7446808510638298


In [11]:
##Methode 2 : jaccard distance en mixant nom et manufacturer
number_of_matches = 0
matches=[]
for i in range(len(company1)):
    tokens1name = nltk.word_tokenize(company1.iloc[i,1] + ' ' + str(company1.iloc[i,3]))
    ng1_tokensname = set(nltk.ngrams(tokens1name, n=1))
    #print(ng1_tokensname)
    for j in range(len(company2)):
        tokens2name = nltk.word_tokenize(company2.iloc[j,1] + ' ' + str(company2.iloc[j,3]))
        ng2_tokensname = set(nltk.ngrams(tokens2name, n=1))
        jd_ng1_ng2_name = nltk.jaccard_distance(ng1_tokensname, ng2_tokensname)
        name_score = nltk.edit_distance(company1.iloc[i,1] , company2.iloc[j,1] )
        if (jd_ng1_ng2_name <= 0.75) : #or name_score<=1 :
            number_of_matches = number_of_matches +1
            matches.append((company1.iloc[i,0],company2.iloc[j,0]))
print("Number of matches: {}".format(number_of_matches))
matches_df = pd.DataFrame(matches)
matches_df.columns= ['idCompany1','idCompany2']
diff_df = pd.merge(ground_truth_matches, matches_df, how='outer', indicator='Exist')
true_positives = diff_df[diff_df.Exist=='both']
false_positives = diff_df[diff_df.Exist=='right_only']
false_negatives = diff_df[diff_df.Exist=='left_only']
print("Number of true positives: {}".format(len(true_positives)))
print("Number of false positives: {}".format(len(false_positives)))
print("Number of false negatives: {}".format(len(false_negatives)))
precision = len(true_positives)/(len(true_positives)+ len(false_positives))
print("Precision: {}".format(precision))
recall = len(true_positives)/(len(true_positives)+ len(false_negatives))
print("Recall: {}".format(recall))
f_measure = 2*(precision*recall)/(precision+recall)
print("F measure: {}".format(f_measure))

Number of matches: 114
Number of true positives: 87
Number of false positives: 27
Number of false negatives: 18
Precision: 0.7631578947368421
Recall: 0.8285714285714286
F measure: 0.7945205479452055


In [101]:
##Methode 3 : idem avec des n-grams de 2
number_of_matches = 0
matches=[]
for i in range(len(company1)):
    tokens1name = nltk.word_tokenize(company1.iloc[i,1] + ' ' + str(company1.iloc[i,3]))
    ng1_tokensname = set(nltk.ngrams(tokens1name, n=2))
    #print(ng1_tokensname)
    for j in range(len(company2)):
        tokens2name = nltk.word_tokenize(company2.iloc[j,1] + ' ' + str(company2.iloc[j,3]))
        ng2_tokensname = set(nltk.ngrams(tokens2name, n=2))
        jd_ng1_ng2_name = nltk.jaccard_distance(ng1_tokensname, ng2_tokensname)
        name_score = nltk.edit_distance(company1.iloc[i,1] , company2.iloc[j,1] )
        if (jd_ng1_ng2_name <= 0.7) : #or name_score<=1 :
            number_of_matches = number_of_matches +1
            matches.append((company1.iloc[i,0],company2.iloc[j,0]))
print("Number of matches: {}".format(number_of_matches))
matches_df = pd.DataFrame(matches)
matches_df.columns= ['idCompany1','idCompany2']
diff_df = pd.merge(ground_truth_matches, matches_df, how='outer', indicator='Exist')
true_positives = diff_df[diff_df.Exist=='both']
false_positives = diff_df[diff_df.Exist=='right_only']
false_negatives = diff_df[diff_df.Exist=='left_only']
print("Number of true positives: {}".format(len(true_positives)))
print("Number of false positives: {}".format(len(false_positives)))
print("Number of false negatives: {}".format(len(false_negatives)))
precision = len(true_positives)/(len(true_positives)+ len(false_positives))
print("Precision: {}".format(precision))
recall = len(true_positives)/(len(true_positives)+ len(false_negatives))
print("Recall: {}".format(recall))
f_measure = 2*(precision*recall)/(precision+recall)
print("F measure: {}".format(f_measure))

Number of matches: 38
Number of true positives: 38
Number of false positives: 0
Number of false negatives: 64
Precision: 1.0
Recall: 0.37254901960784315
F measure: 0.5428571428571428


In [89]:
##Methode 4 : idem avec des n-grams de 3
number_of_matches = 0
matches=[]
for i in range(len(company1)):
    tokens1name = nltk.word_tokenize(company1.iloc[i,1] + ' ' + str(company1.iloc[i,3]))
    ng1_tokensname = set(nltk.ngrams(tokens1name, n=3))
    #print(ng1_tokensname)
    for j in range(len(company2)):
        tokens2name = nltk.word_tokenize(company2.iloc[j,1] + ' ' + str(company2.iloc[j,3]))
        ng2_tokensname = set(nltk.ngrams(tokens2name, n=3))
        jd_ng1_ng2_name = nltk.jaccard_distance(ng1_tokensname, ng2_tokensname)
        name_score = nltk.edit_distance(company1.iloc[i,1] , company2.iloc[j,1] )
        if (jd_ng1_ng2_name <= 0.75) : #or name_score<=1 :
            number_of_matches = number_of_matches +1
            matches.append((company1.iloc[i,0],company2.iloc[j,0]))
print("Number of matches: {}".format(number_of_matches))
matches_df = pd.DataFrame(matches)
matches_df.columns= ['idCompany1','idCompany2']
diff_df = pd.merge(ground_truth_matches, matches_df, how='outer', indicator='Exist')
true_positives = diff_df[diff_df.Exist=='both']
false_positives = diff_df[diff_df.Exist=='right_only']
false_negatives = diff_df[diff_df.Exist=='left_only']
print("Number of true positives: {}".format(len(true_positives)))
print("Number of false positives: {}".format(len(false_positives)))
print("Number of false negatives: {}".format(len(false_negatives)))
precision = len(true_positives)/(len(true_positives)+ len(false_positives))
print("Precision: {}".format(precision))
recall = len(true_positives)/(len(true_positives)+ len(false_negatives))
print("Recall: {}".format(recall))
f_measure = 2*(precision*recall)/(precision+recall)
print("F measure: {}".format(f_measure))

Number of matches: 25
Number of true positives: 25
Number of false positives: 0
Number of false negatives: 75
Precision: 1.0
Recall: 0.25
F measure: 0.4


In [109]:
##Methode 4 : jaccard distance en mixant nom, description et manufacturer
number_of_matches = 0
matches=[]
for i in range(len(company1)):
    tokens1name = nltk.word_tokenize(company1.iloc[i,1] + ' ' + str(company1.iloc[i,2]) + ' ' + str(company1.iloc[i,3]))
    ng1_tokensname = set(nltk.ngrams(tokens1name, n=1))
    #print(ng1_tokensname)
    for j in range(len(company2)):
        tokens2name = nltk.word_tokenize(company2.iloc[j,1] + ' ' + str(company2.iloc[j,2]) + ' ' + str(company2.iloc[j,3]))
        ng2_tokensname = set(nltk.ngrams(tokens2name, n=1))
        jd_ng1_ng2_name = nltk.jaccard_distance(ng1_tokensname, ng2_tokensname)
        name_score = nltk.edit_distance(company1.iloc[i,1] , company2.iloc[j,1] )
        if (jd_ng1_ng2_name <= 0.8) : #or name_score<=1 :
            number_of_matches = number_of_matches +1
            matches.append((company1.iloc[i,0],company2.iloc[j,0]))
print("Number of matches: {}".format(number_of_matches))
matches_df = pd.DataFrame(matches)
matches_df.columns= ['idCompany1','idCompany2']
diff_df = pd.merge(ground_truth_matches, matches_df, how='outer', indicator='Exist')
true_positives = diff_df[diff_df.Exist=='both']
false_positives = diff_df[diff_df.Exist=='right_only']
false_negatives = diff_df[diff_df.Exist=='left_only']
print("Number of true positives: {}".format(len(true_positives)))
print("Number of false positives: {}".format(len(false_positives)))
print("Number of false negatives: {}".format(len(false_negatives)))
precision = len(true_positives)/(len(true_positives)+ len(false_positives))
print("Precision: {}".format(precision))
recall = len(true_positives)/(len(true_positives)+ len(false_negatives))
print("Recall: {}".format(recall))
f_measure = 2*(precision*recall)/(precision+recall)
print("F measure: {}".format(f_measure))

Number of matches: 38
Number of true positives: 32
Number of false positives: 6
Number of false negatives: 70
Precision: 0.8421052631578947
Recall: 0.3137254901960784
F measure: 0.45714285714285713


In [205]:
##Methode 5 : methode 2 + price ratio<3
number_of_matches = 0
matches=[]
for i in range(len(company1)):
    try :  
        price1 = float(company1.iloc[i,4]) 
    except : 
        price1 = 0
    tokens1name = nltk.word_tokenize(str(company1.iloc[i,3]) + ' ' + company1.iloc[i,1])
    ng1_tokensname = set(nltk.ngrams(tokens1name, n=1))
    #print(ng1_tokensname)
    for j in range(len(company2)):
        try :  
            price2 = float(company2.iloc[j,4]) 
        except : 
            price2 = 0
        if price1* price2 == 0:
            price_ratio=1
        else:
            price_ratio =max(price1, price2)/min(price1, price2)
        tokens2name = nltk.word_tokenize(str(company2.iloc[j,3]) + ' ' + company2.iloc[j,1]  )
        ng2_tokensname = set(nltk.ngrams(tokens2name, n=1))
        jd_ng1_ng2_name = nltk.jaccard_distance(ng1_tokensname, ng2_tokensname)
        name_score = nltk.edit_distance(company1.iloc[i,1] , company2.iloc[j,1] )
        if ((jd_ng1_ng2_name <= 0.65) and price_ratio<3) or ((jd_ng1_ng2_name <= 0.75) and price_ratio<1.25): #or name_score<=1 :
            number_of_matches = number_of_matches +1
            matches.append((company1.iloc[i,0],company2.iloc[j,0]))
print("Number of matches: {}".format(number_of_matches))
matches_df = pd.DataFrame(matches)
matches_df.columns= ['idCompany1','idCompany2']
diff_df = pd.merge(ground_truth_matches, matches_df, how='outer', indicator='Exist')
true_positives = diff_df[diff_df.Exist=='both']
false_positives = diff_df[diff_df.Exist=='right_only']
false_negatives = diff_df[diff_df.Exist=='left_only']
print("Number of true positives: {}".format(len(true_positives)))
print("Number of false positives: {}".format(len(false_positives)))
print("Number of false negatives: {}".format(len(false_negatives)))
precision = len(true_positives)/(len(true_positives)+ len(false_positives))
print("Precision: {}".format(precision))
recall = len(true_positives)/(len(true_positives)+ len(false_negatives))
print("Recall: {}".format(recall))
f_measure = 2*(precision*recall)/(precision+recall)
print("F measure: {}".format(f_measure))

Number of matches: 91
Number of true positives: 81
Number of false positives: 10
Number of false negatives: 24
Precision: 0.8901098901098901
Recall: 0.7714285714285715
F measure: 0.8265306122448981


In the above example, the two records share the same value for attributes 'name' and 'address'. However, they have slightly different values for the columns 'city' and 'cuisine'

In [218]:
f=8
company1[company1['id']==false_negatives.iloc[f,0]]



,id,title,description,manufacturer,price
27,b000bnkbdm,sound studio 3: record edit add effects to aud...,sound studio 3 is an easy-to-use mac os x appl...,freeverse software,79.99


In [219]:
company2[company2['id']==false_negatives.iloc[f,1]]

,id,name,description,manufacturer,price
78,18374143831267894200,sound studio 3 for mac,easytouse mac os x application for recording a...,NaN,79.99


In [206]:
tokens2name 

['nan',
 'weekly',
 'reader',
 'preparing',
 'for',
 'kindergarten',
 '2008',
 '(',
 'pc/mac',
 ')',
 'fogware']

In the above example, on the other hand, the two records are associated with different names, cities and cuisiones.

This file represents a simple example of datasets, on which we can experiment with th etechniques presented in the course to try identify duplicates, without using (that is relying on the values of) the column "unique_id".

In [ ]:
# We start by adding a new column to identify the records (lines) in our dataframe
df_restaurants.insert(0,'record_ID', range(0, len(df_restaurants)))

In [ ]:
df_restaurants.head(5)

Exhaustive comparisons: every record is compared with every other record

We start by applying an exhaustive strategy whereby every record in the CSV file, is compared with every other record. 

The code below does this for us. In doing so, it uses the following rule:

For two records to match, i.e. refer to the same restaurant in the real world:
* The edit distance between the attribute name values of the two records needs to be smaller or equal to 3, and 
* they need to have the same value for the cuisine attribute.

In [ ]:
df_restaurants[df_restaurants.record_ID.isin([43, 622])]

In [ ]:
num_records = len(df_restaurants)
matches = []
matchescomplet = []

number_of_matches = 0
tokens1=[]
tokens2=[]
start = time.process_time()
for i in range(0,num_records):
    
    # Après tokenization , calcul du ngrams (n=1) pour le name qui servira pour la Jaccard distance, pour la ligne i
    tokens1name = nltk.word_tokenize(df_restaurants.iloc[i,1]) 
    ng1_tokensname = set(nltk.ngrams(tokens1name, n=1))
    
    # Après tokenization , calcul du ngrams (n=1) pour l'adresse qui servira pour la Jaccard distance,, pour la ligne i
    tokens1adr = nltk.word_tokenize(df_restaurants.iloc[i,2]) 
    ng1_tokensadr = set(nltk.ngrams(tokens1adr, n=1))
    
    
    for j in range(i+1,num_records):
        
        # Après tokenization , calcul du ngrams (n=1) pour le name qui servira pour la Jaccard distance, , pour la ligne j
        tokens2name = nltk.word_tokenize( df_restaurants.iloc[j,1]) 
        ng2_tokensname = set(nltk.ngrams(tokens2name, n=1))
        
        # Après tokenization , calcul du ngrams (n=1) pour le name qui servira pour la Jaccard distance, , pour la ligne j
        tokens2adr = nltk.word_tokenize( df_restaurants.iloc[j,2]) 
        ng2_tokensadr = set(nltk.ngrams(tokens2adr, n=1))
       
        # calcul de la Jaccard distance pour le name entre la ligne i et la ligne j ("item based" avec ngrams (n=1)) 
        jd_ng1_ng2_name = nltk.jaccard_distance(ng1_tokensname, ng2_tokensname)  # jaccard distance entre les ngram=1 des names
        
        # calcul de la Jaccard distance pour l'adresse entre la ligne i et la ligne j ("item based" avec ngrams (n=1)) 
        jd_ng1_ng2_adr = nltk.jaccard_distance(ng1_tokensadr, ng2_tokensadr)  # jaccard distance entre les ngram=1 des adresses
    
        # Rule for matching: 
        # disjonction entre une similarité entre les names (name_score<=1) 
        # et une similarité conjugée entre les adresses et les noms (jd_ng1_ng2_adr <= 0.6 and jd_ng1_ng2_name <= 0.6)
        name_score = nltk.edit_distance(df_restaurants.iloc[i,1], df_restaurants.iloc[j,1])
        
        # Rule for matching: Distance between names is smaller or equal to 3 and the cuisine is the same 
        if (jd_ng1_ng2_adr <= 0.6 and jd_ng1_ng2_name <= 0.6) or name_score<=1 :
            number_of_matches = number_of_matches +1 
            # matchescomplet.append((df_restaurants.iloc[i,0],df_restaurants.iloc[i,1], df_restaurants.iloc[i,2],df_restaurants.iloc[i,5], df_restaurants.iloc[j,0],df_restaurants.iloc[j,1], df_restaurants.iloc[j,2],df_restaurants.iloc[j,5]))
            matches.append((df_restaurants.iloc[i,0],df_restaurants.iloc[j,0]))

end = time.process_time()

print("Number of matches: {}".format(number_of_matches))
print("Processing time: {}".format(end - start))
for _ in matchescomplet:
     print(_)

In [ ]:
# quelques tests pour ajuster les critères de notre algorithme
name_score = nltk.edit_distance(df_restaurants.iloc[73,1], df_restaurants.iloc[763,1])
name_score   # 11
# comme on le voit ci-dessous, la différence est le mot "restaurant", l'edit distance est très importante (11), 
# on ne peut pas se baser dessus pour dire que c le même resto, il faut qu'on ajoute un critère "item based" 
# en plus du critère edit_distance name_score<=1

In [ ]:
# qq tests pour ajuster les critères de notre alogorithme
df_restaurants[df_restaurants.record_ID.isin([73, 763])]

In [ ]:
# name_score = nltk.edit_distance(df_restaurants.iloc[32,1], df_restaurants.iloc[759,1])
#print(name_score)
tokens1 = nltk.word_tokenize(df_restaurants.iloc[73,1]) 
tokens2 = nltk.word_tokenize( df_restaurants.iloc[763,1]) 
print(tokens1)
print(tokens2)
ng1_tokens = set(nltk.ngrams(tokens1, n=1))
ng2_tokens = set(nltk.ngrams(tokens2, n=1))
print(ng1_tokens)
print(ng2_tokens)

jd_sent_1_2 = nltk.jaccard_distance(ng1_tokens, ng2_tokens)
print(jd_sent_1_2)
# jd_ng1_ng2_adr <= 0.6,ce seuil de 0.6 suffira dire que les lignes 32 et 759 sont le même restaurant

In [ ]:
# adresse
#print(name_score)
tokens1 = nltk.word_tokenize(df_restaurants.iloc[73,2]) 
tokens2 = nltk.word_tokenize( df_restaurants.iloc[763,2]) 
print(tokens1)
print(tokens2)
ng1_tokens = set(nltk.ngrams(tokens1, n=1))
ng2_tokens = set(nltk.ngrams(tokens2, n=1))
print(ng1_tokens)
print(ng2_tokens)

jd_sent_1_2 = nltk.jaccard_distance(ng1_tokens, ng2_tokens)
print(jd_sent_1_2)
# ça ne passe pas , mais c pas grave car mettre le seuil à 0.67 va nous rajouter beaucoup de faux positifs 
# on a testé ce seuil plus élevé de 0.67

In [ ]:
name_score = nltk.edit_distance(df_restaurants.iloc[6,1], df_restaurants.iloc[754,1])
name_score

In [ ]:
# name_score = nltk.edit_distance(df_restaurants.iloc[32,1], df_restaurants.iloc[759,1])
#print(name_score)
tokens1 = nltk.word_tokenize(df_restaurants.iloc[6,1]) 
tokens2 = nltk.word_tokenize( df_restaurants.iloc[754,1]) 
print(tokens1)
print(tokens2)
ng1_tokens = set(nltk.ngrams(tokens1, n=1))
ng2_tokens = set(nltk.ngrams(tokens2, n=1))
print(ng1_tokens)
print(ng2_tokens)

jd_sent_1_2 = nltk.jaccard_distance(ng1_tokens, ng2_tokens)
print(jd_sent_1_2)

In [ ]:
name_score = nltk.edit_distance(df_restaurants.iloc[6,1], df_restaurants.iloc[754,1])
name_score

In [ ]:
# name_score = nltk.edit_distance(df_restaurants.iloc[32,1], df_restaurants.iloc[759,1])
#print(name_score)
tokens1 = nltk.word_tokenize(df_restaurants.iloc[32,1]) 
tokens2 = nltk.word_tokenize( df_restaurants.iloc[759,1]) 
print(tokens1)
print(tokens2)
ng1_tokens = set(nltk.ngrams(tokens1, n=1))
ng2_tokens = set(nltk.ngrams(tokens2, n=1))
print(ng1_tokens)
print(ng2_tokens)

jd_sent_1_2 = nltk.jaccard_distance(ng1_tokens, ng2_tokens)
print(jd_sent_1_2)

In [ ]:
# Display results
for match in matches:
    print("The following records {} and {} match".format(match[0],match[1]))
    print("The restaurants with the following names {} and {} match.".format(df_restaurants.iloc[match[0],1],df_restaurants.iloc[match[1],1]))
    print("The restaurants with the following addresses {} and {} match.".format(df_restaurants.iloc[match[0],2],df_restaurants.iloc[match[1],2]))
    print("\n")

Note that the rule applied in the above code is not great. You may want to try other kind of distances, other thresholds, and other rules to identify matches.

# Assessing the quality of the results

To do so, we first need to compute the ground truth (that is the list of correct matches) considering the attribute unique_id.

In [ ]:
ground_truth_matches = pd.read_csv("./restaurants.csv")

In [ ]:
ground_truth_matches.insert(0, 'record_ID', range(0, len(ground_truth_matches)))

In [ ]:
ground_truth_matches.head(5)

In [ ]:
ground_truth_matches = pd.merge(ground_truth_matches,
                                ground_truth_matches,
                                on = 'unique_id')

In [ ]:
ground_truth_matches.head(5)

In [ ]:
len(ground_truth_matches)

In [ ]:
ground_truth_matches = ground_truth_matches.query('record_ID_x < record_ID_y')

In [ ]:
ground_truth_matches.head(20)

In [ ]:
ground_truth_matches = ground_truth_matches[['record_ID_x','record_ID_y']]

In [ ]:
print(ground_truth_matches)

In [ ]:
print(len(ground_truth_matches))

In [ ]:
print(len(matches))


In [ ]:
matches_df = pd.DataFrame(matches)
matches_df.head(5)

In [ ]:
matches_df = pd.DataFrame(matches)
matches_df.columns= ['record_ID_x','record_ID_y']

In [ ]:
# on s'assure que les couples record_ID_x et record_ID_y sont dans le bons sens (record_ID_x < record_ID_y)
# comme dans ground_truth
matches_df[matches_df['record_ID_x'] >= matches_df['record_ID_y'] ]
# 0 lignes trouvées , donc c OK.


In [ ]:
matches_df.head()

In [ ]:
diff_df = pd.merge(ground_truth_matches, matches_df, how='outer', indicator='Exist')

In [ ]:
diff_df.head(5)

In [ ]:
true_positives = diff_df[diff_df.Exist=='both']
false_positives = diff_df[diff_df.Exist=='right_only']
false_negatives = diff_df[diff_df.Exist=='left_only']

In [41]:
# les vrais duplicats que notre algo a pu détecter
true_positives.head()

,idCompany1,idCompany2,Exist
0,b0002ibev4,1887899244694755891,both
1,b0007lw22g,18398718226932431716,both
5,b0001wn16m,10092468528845066077,both
7,b000ozhfsq,18438075297130458214,both
9,b00006sijr,6247936198343071793,both


In [ ]:
#Example of a true positive
df_restaurants[df_restaurants.record_ID.isin(['6','754'])]

In [42]:
# notre algo les a sortis comme restos en double mais c pas vrai
false_positives.head()

,idCompany1,idCompany2,Exist
103,b0000c6fjm,18394964067436310447,right_only
104,b0000c6fjm,10092468528845066077,right_only
105,b000ndibq4,9761533219806554318,right_only
106,b000o27t8a,9761533219806554318,right_only
107,b000o27t8a,14054232840925252286,right_only


In [ ]:
# notre critère de duplicate :
# (jd_ng1_ng2_adr <= 0.6 and jd_ng1_ng2_name <= 0.6) or (name_score<=1 and jd_ng1_ng2_adr <= 0.6) 
# eliminer grace jd_ng1_ng2_adr = 0.6666
df_restaurants[df_restaurants.record_ID.isin(['55','56'])]
# le name est le même donc l'algo dit que ce le même restaurant alors que ce n'est pas vrai.

In [ ]:
# pareil c pas le même resto alors que l'algo les a retenu comme duplicate
# car les names diffèrent d'un seul caractère.
df_restaurants[df_restaurants.record_ID.isin(['87','88'])]

In [ ]:
# name_score<=1
name_score = nltk.edit_distance(df_restaurants.iloc[87,1], df_restaurants.iloc[88,1])
name_score

In [43]:
# les vrais duplicates que l'algo n'a pas détecté
false_negatives.head()

,idCompany1,idCompany2,Exist
2,b000ofnri8,12244614697089679523,left_only
3,b0007d8r5k,13775362651326388438,left_only
4,b00099qrok,9755705822363275907,left_only
6,b0002e3g6o,18384557845547191313,left_only
8,b0009yx9be,1021042895134770712,left_only


In [ ]:
df_restaurants[df_restaurants.record_ID.isin(['32','759'])]

In [ ]:
# faux négatif
# pour l'algo le 32 et le 759 c'est pas le même restaurant, pourtant c le même
# en effet les names diffèrents en lettres et en mots : 
# name_score > 1 et jd_ng1_ng2_name > 0.6 (ça suffit pour l'algo pour l'éliminer ) et en plus jd_ng1_ng2_adr > 0.6
name_score = nltk.edit_distance(df_restaurants.iloc[32,1], df_restaurants.iloc[759,1])
name_score

In [ ]:
 # (jd_ng1_ng2_adr <= 0.6) and jd_ng1_ng2_name <= 0.6) or (name_score<=1)
    
# name_score = nltk.edit_distance(df_restaurants.iloc[32,1], df_restaurants.iloc[759,1])

tokens1 = nltk.word_tokenize(df_restaurants.iloc[32,1])   # name
tokens2 = nltk.word_tokenize( df_restaurants.iloc[759,1]) 
print(tokens1)
print(tokens2)
ng1_tokens = set(nltk.ngrams(tokens1, n=1))
ng2_tokens = set(nltk.ngrams(tokens2, n=1))
print(ng1_tokens)
print(ng2_tokens)

jd_sent_1_2 = nltk.jaccard_distance(ng1_tokens, ng2_tokens)
print(jd_sent_1_2)

In [ ]:
 # (jd_ng1_ng2_adr <= 0.6) and (name_score<=2 or jd_ng1_ng2_name <= 0.67)
    
# name_score = nltk.edit_distance(df_restaurants.iloc[32,1], df_restaurants.iloc[759,1])
#print(name_score)
tokens1 = nltk.word_tokenize(df_restaurants.iloc[73,2])   # adresse 
tokens2 = nltk.word_tokenize( df_restaurants.iloc[763,2]) 
print(tokens1)
print(tokens2)
ng1_tokens = set(nltk.ngrams(tokens1, n=1))
ng2_tokens = set(nltk.ngrams(tokens2, n=1))
print(ng1_tokens)
print(ng2_tokens)

jd_sent_1_2 = nltk.jaccard_distance(ng1_tokens, ng2_tokens)
print(jd_sent_1_2)

In [ ]:
print(len(ground_truth_matches))
print(len(matches_df))
print(len(true_positives) , 'true_positives')
print(len(false_positives) ,'false_positives')
print(len(false_negatives)  , 'false_negatives')

# len(true_positives)  +  len(false_negatives) = len(ground_truth_matches)

# len(matches_df)) - len(false_positif) + len(false_negatives)     = ground_truth_matches

In [ ]:
precision = len(true_positives)/(len(true_positives)+ len(false_positives))
print(precision)

Note that if you are using pyton 2.7 (instead of Python 3), you would need to convert integers to float prior to performing the division

In [ ]:
recall = len(true_positives)/(len(true_positives)+ len(false_negatives))
print(recall)

In [ ]:
f_measure = 2*(precision*recall)/(precision+recall)
print(f_measure)

# Windowing (SNM) method

In [ ]:
df_restaurants.head()

In [ ]:
# 841 842
# qq tests pour choisir sur quel champ on va faire le sort 
# le sorted name parait intéressant
df_restaurants.sort_values(by=['name']).head(20)

### Le tri est fait dans ce qui suit selon le champ "name"

In [ ]:
window = 50   # 

# tri par name car c ce qui permet d'avoir des resto en double les plus proches possibles 

df_restaurants= df_restaurants.sort_values(by=['name'])  

number_of_matchesw = 0
num_records = len(df_restaurants)
matchesw = []
matchescompletw = []

start = time.process_time()
for i in range(0,min(window,len(df_restaurants))):
    
    tokens1name = nltk.word_tokenize(df_restaurants.iloc[i,1]) 
    ng1_tokensname = set(nltk.ngrams(tokens1name, n=1))
    
    tokens1adr = nltk.word_tokenize(df_restaurants.iloc[i,2]) 
    ng1_tokensadr = set(nltk.ngrams(tokens1adr, n=1))
    
    
    for j in range(i+1,min(window,len(df_restaurants))):
        tokens2name = nltk.word_tokenize( df_restaurants.iloc[j,1]) 
        ng2_tokensname = set(nltk.ngrams(tokens2name, n=1))
        
        
        tokens2adr = nltk.word_tokenize( df_restaurants.iloc[j,2]) 
        ng2_tokensadr = set(nltk.ngrams(tokens2adr, n=1))
#         print(tokens1)
#         print(tokens2)       
        jd_ng1_ng2_name = nltk.jaccard_distance(ng1_tokensname, ng2_tokensname)  # jaccard distance entre les ngram=1 des names
        jd_ng1_ng2_adr = nltk.jaccard_distance(ng1_tokensadr, ng2_tokensadr)  # jaccard distance entre les ngram=1 des adresses
    
        name_score = nltk.edit_distance(df_restaurants.iloc[i,1], df_restaurants.iloc[j,1])
        
        # Rule for matching: Distance between names is smaller or equal to 3 and the cuisine is the same 
        if (jd_ng1_ng2_adr <= 0.6 and jd_ng1_ng2_name <= 0.6) or name_score<=1 :
            number_of_matchesw = number_of_matchesw +1 
            # matchescomplet.append((df_restaurants.iloc[i,0],df_restaurants.iloc[i,1], df_restaurants.iloc[i,2],df_restaurants.iloc[i,5], df_restaurants.iloc[j,0],df_restaurants.iloc[j,1], df_restaurants.iloc[j,2],df_restaurants.iloc[j,5]))
            matchesw.append((df_restaurants.iloc[i,0],df_restaurants.iloc[j,0]))
            matchescompletw.append((df_restaurants.iloc[i,0],df_restaurants.iloc[i,1], df_restaurants.iloc[i,2],df_restaurants.iloc[i,5], df_restaurants.iloc[j,0],df_restaurants.iloc[j,1], df_restaurants.iloc[j,2],df_restaurants.iloc[j,5]))
                     
            
            
for i in range(window,len(df_restaurants)):
    
    tokens1name = nltk.word_tokenize(df_restaurants.iloc[i,1]) 
    ng1_tokensname = set(nltk.ngrams(tokens1name, n=1))
    
    tokens1adr = nltk.word_tokenize(df_restaurants.iloc[i,2]) 
    ng1_tokensadr = set(nltk.ngrams(tokens1adr, n=1))
    
    
    for j in range(i-window+1,i):
        tokens2name = nltk.word_tokenize( df_restaurants.iloc[j,1]) 
        ng2_tokensname = set(nltk.ngrams(tokens2name, n=1))
        
        
        tokens2adr = nltk.word_tokenize( df_restaurants.iloc[j,2]) 
        ng2_tokensadr = set(nltk.ngrams(tokens2adr, n=1))
     
        jd_ng1_ng2_name = nltk.jaccard_distance(ng1_tokensname, ng2_tokensname)  # jaccard distance entre les ngram=1 des names
        jd_ng1_ng2_adr = nltk.jaccard_distance(ng1_tokensadr, ng2_tokensadr)  # jaccard distance entre les ngram=1 des adresses
    
        name_score = nltk.edit_distance(df_restaurants.iloc[i,1], df_restaurants.iloc[j,1])
        
        # Rule for matching: Distance between names is smaller or equal to 3 and the cuisine is the same 
        if (jd_ng1_ng2_adr <= 0.6 and jd_ng1_ng2_name <= 0.6) or name_score<=1 :
            number_of_matchesw = number_of_matchesw +1 
            # matchescomplet.append((df_restaurants.iloc[i,0],df_restaurants.iloc[i,1], df_restaurants.iloc[i,2],df_restaurants.iloc[i,5], df_restaurants.iloc[j,0],df_restaurants.iloc[j,1], df_restaurants.iloc[j,2],df_restaurants.iloc[j,5]))
            matchesw.append((df_restaurants.iloc[i,0],df_restaurants.iloc[j,0]))
            matchescompletw.append((df_restaurants.iloc[i,0],df_restaurants.iloc[i,1], df_restaurants.iloc[i,2],df_restaurants.iloc[i,5], df_restaurants.iloc[j,0],df_restaurants.iloc[j,1], df_restaurants.iloc[j,2],df_restaurants.iloc[j,5]))
            
end = time.process_time()

print("Number of matches: {}".format(number_of_matchesw))
print("Processing time: {}".format(end - start))            
for _ in matchescompletw:
     print(_)  
# for _ in matches:
#      print(_)          

In [ ]:
# Display results
for match in matchesw:
    print("The following records {} and {} match".format(match[0],match[1]))
    print("The restaurants with the following names {} and {} match.".format(df_restaurants.iloc[match[0],1],df_restaurants.iloc[match[1],1]))
    print("The restaurants with the following addresses {} and {} match.".format(df_restaurants.iloc[match[0],2],df_restaurants.iloc[match[1],2]))
    print("\n")

In [ ]:
matchesw_df = pd.DataFrame(matchesw)
matchesw_df.columns= ['record_ID_x','record_ID_y']

matchesw_df['MIN'] = matchesw_df[['record_ID_x','record_ID_y']].min(axis=1)
matchesw_df['MAX'] = matchesw_df[['record_ID_x','record_ID_y']].max(axis=1)
matchesw_df=matchesw_df[['MIN','MAX']]
matchesw_df.columns=['record_ID_x','record_ID_y']
matchesw_df


diffw_df = pd.merge(ground_truth_matches, matchesw_df, how='outer', indicator='Exist')
true_positivesw = diffw_df[diffw_df.Exist=='both']
false_positivesw = diffw_df[diffw_df.Exist=='right_only']
false_negativesw = diffw_df[diffw_df.Exist=='left_only']
precisionw = len(true_positivesw)/(len(true_positivesw)+ len(false_positivesw))
print(precisionw)
recallw = len(true_positivesw)/(len(true_positivesw)+ len(false_negativesw))
print(recallw)
f_measurew = 2*(precisionw*recallw)/(precisionw+recallw)
print(f_measurew)

In [ ]:
print(len(ground_truth_matches))
print(len(matchesw_df))
print(len(true_positivesw))
print(len(false_positivesw))
print(len(false_negativesw))  
# len(true_positives)  +  len(false_negatives) = len(ground_truth_matches)
# len(matches_df)) - len(false_positif) + len(false_negatives)     = ground_truth_matches

It is worth noting that in the above code, we do not implement the SNM algorithm in its entirety. In particular, we do not implement the last phase of inferring matches using transitivity

# Blocking method

In [ ]:
df_restaurants = pd.read_csv("./restaurants.csv")

In [ ]:
len(df_restaurants)

In [ ]:
df_restaurants.head()

In [ ]:
# We start by adding a new column to identify the records (lines) in our dataframe
df_restaurants.insert(0,'record_ID', range(0, len(df_restaurants)))

In [ ]:
# The blocks correspond to resturants that are located in the same citydf_restaurants.loc[df_restaurants['city']==' atlanta']
df_restaurants.loc[df_restaurants['city']==' atlanta']

In [ ]:
df_restaurants.loc[df_restaurants['city'].str.strip()=='atlanta']

In [ ]:
# on va créer un dict "df_restov" des restaurants de chaque ville
# pour une clé= ville, la valeur du dict serait égale à un dataframe représentant les restos de cette ville
df_restov= {}
for ville in df_restaurants['city'].unique():
    
    df_restov[ville]   = df_restaurants.loc[df_restaurants['city']==ville]
    num_records = len(df_restov[ville])
    print(ville)   # on affiche la ville
    print(num_records) # on affiche le nombre de restos par ville



In [ ]:
# on vérifie  pour atlanta que ça marche bien, on a bien le dataframe qu'on voudrait.
print(type(df_restov[" atlanta"]))
print(df_restov[" atlanta"])


In [ ]:
df_restov[" atlanta"].head()

In [ ]:
# on testel'algo précédent sur juste un dataframe celui des restos de " atlanta"  (avec un espace devant)
num_records = len(df_restov[" atlanta"])
amatches = []
amatchescomplet = []

anumber_of_matches = 0
tokens1=[]
tokens2=[]
start = time.process_time()
for i in range(0,num_records):
    
    # Après tokenization , calcul du ngrams (n=1) pour le name qui servira pour la Jaccard distance, pour la ligne i
    tokens1name = nltk.word_tokenize(df_restov[" atlanta"].iloc[i,1]) 
    ng1_tokensname = set(nltk.ngrams(tokens1name, n=1))
    
    # Après tokenization , calcul du ngrams (n=1) pour l'adresse qui servira pour la Jaccard distance,, pour la ligne i
    tokens1adr = nltk.word_tokenize(df_restov[" atlanta"].iloc[i,2]) 
    ng1_tokensadr = set(nltk.ngrams(tokens1adr, n=1))
    
    
    for j in range(i+1,num_records):
        
        # Après tokenization , calcul du ngrams (n=1) pour le name qui servira pour la Jaccard distance, , pour la ligne j
        tokens2name = nltk.word_tokenize( df_restov[" atlanta"].iloc[j,1]) 
        ng2_tokensname = set(nltk.ngrams(tokens2name, n=1))
        
        # Après tokenization , calcul du ngrams (n=1) pour le name qui servira pour la Jaccard distance, , pour la ligne j
        tokens2adr = nltk.word_tokenize( df_restov[" atlanta"].iloc[j,2]) 
        ng2_tokensadr = set(nltk.ngrams(tokens2adr, n=1))
     
        # calcul de la Jaccard distance pour le name entre la ligne i et la ligne j ("item based" avec ngrams (n=1)) 
        jd_ng1_ng2_name = nltk.jaccard_distance(ng1_tokensname, ng2_tokensname)
        
        # calcul de la Jaccard distance pour l'adresse entre la ligne i et la ligne j ("item based" avec ngrams (n=1)) 
        jd_ng1_ng2_adr = nltk.jaccard_distance(ng1_tokensadr, ng2_tokensadr)  
    
        name_score = nltk.edit_distance(df_restov[" atlanta"].iloc[i,1], df_restov[" atlanta"].iloc[j,1])
        
        # Rule for matching: 
        # disjonction entre une similarité entre les names (name_score<=1) 
        # et une similarité conjugée entre les adresses et les noms (jd_ng1_ng2_adr <= 0.6 and jd_ng1_ng2_name <= 0.6)
        if (jd_ng1_ng2_adr <= 0.6 and jd_ng1_ng2_name <= 0.6) or name_score<=1 :
            anumber_of_matches = anumber_of_matches +1 
            matchescomplet.append((df_restov[" atlanta"].iloc[i,0],df_restov[" atlanta"].iloc[i,1], \
            df_restov[" atlanta"].iloc[i,2],df_restov[" atlanta"].iloc[i,3], df_restov[" atlanta"].iloc[i,5], \
            df_restov[" atlanta"].iloc[j,0],df_restov[" atlanta"].iloc[j,1], df_restov[" atlanta"].iloc[j,2], \
            df_restov[" atlanta"].iloc[j,3],df_restov[" atlanta"].iloc[j,5]))
            amatches.append((df_restov[" atlanta"].iloc[i,0],df_restov[" atlanta"].iloc[j,0]))

end = time.process_time()

print("Number of matches: {}".format(anumber_of_matches))
print("Processing time: {}".format(end - start))
for _ in amatchescomplet:
     print(_)

In [ ]:
# nous allons refaire le dict mais en éliminant les espaces saisis avant et après chaque ville
# par précaution pour éviter des villes en double
# et nous allons imprimer le nombre de restos par ville.

df_restov={}
cumul= 0
# il faut enlever les espaces au début et à la fin de chaque ville dans le dataframe, 
# sinon on va rater des restos en double car ils ne seront pas dans le même block.

for ville in df_restaurants['city'].str.strip().unique():   
     print(ville)
     df_restov[ville]   = df_restaurants.loc[df_restaurants['city'].str.strip()==ville]
     print(len(df_restov[ville]))
     cumul += len(df_restov[ville])

print(cumul)
# on vérifie qu'on retrouve bien un total de 865 restaurants.

In [ ]:
# Généralisation de la BLOCKING METHOD à toutes les villes 
bmatches = []
bmatchescomplet = []
bnumber_of_matches = 0
start = time.process_time()
    
for ville in df_restaurants['city'].str.strip().unique():
        # affichage de la ville et du nombre de restos par ville
        # pour les matcher entre eux
        print(ville)  
        num_records = len(df_restov[ville])
        print(num_records)
        
        tokens1=[]
        tokens2=[]
       
        for i in range(0,num_records):

            tokens1name = nltk.word_tokenize(df_restov[ville].iloc[i,1]) 
            ng1_tokensname = set(nltk.ngrams(tokens1name, n=1))

            tokens1adr = nltk.word_tokenize(df_restov[ville].iloc[i,2]) 
            ng1_tokensadr = set(nltk.ngrams(tokens1adr, n=1))


            for j in range(i+1,num_records):

                tokens2name = nltk.word_tokenize( df_restov[ville].iloc[j,1]) 
                ng2_tokensname = set(nltk.ngrams(tokens2name, n=1))


                tokens2adr = nltk.word_tokenize( df_restov[ville].iloc[j,2]) 
                ng2_tokensadr = set(nltk.ngrams(tokens2adr, n=1))

                jd_ng1_ng2_name = nltk.jaccard_distance(ng1_tokensname, ng2_tokensname)  # jaccard distance entre les ngram=1 des names
                jd_ng1_ng2_adr = nltk.jaccard_distance(ng1_tokensadr, ng2_tokensadr)  # jaccard distance entre les ngram=1 des adresses

                name_score = nltk.edit_distance(df_restov[ville].iloc[i,1], df_restov[ville].iloc[j,1])

                # Rule for matching: Item based Jaccard Distance with ngram=1 between adresses and between names or edit distance between names 
                if (jd_ng1_ng2_adr <= 0.6 and jd_ng1_ng2_name <= 0.6) or name_score<=1 :
                    bnumber_of_matches = bnumber_of_matches +1 
                    bmatchescomplet.append((df_restov[ville].iloc[i,0],df_restov[ville].iloc[i,1], \
                    df_restov[ville].iloc[i,2],df_restov[ville].iloc[i,3], df_restov[ville].iloc[i,5], \
                    df_restov[ville].iloc[j,0],df_restov[ville].iloc[j,1], df_restov[ville].iloc[j,2], \
                    df_restov[ville].iloc[j,3],df_restov[ville].iloc[j,5]))
                    bmatches.append((df_restov[ville].iloc[i,0],df_restov[ville].iloc[j,0]))

end = time.process_time()

print("Number of matches: {}".format(bnumber_of_matches))
print("Processing time: {}".format(end - start))
# for _ in matchescomplet:
#        print(_)
   

#### Rappel des résultats de l'algo original sans blocking:
####  Number of matches: 127
#### Processing time: 167.984375

#### les infos de l'algo avec  blocking ci-dessus
#### Number of matches: 67
#### Processing time: 25.6875



In [ ]:
import pandas as pd
ground_truth_matches = pd.read_csv("./restaurants.csv")
len(ground_truth_matches)

In [ ]:
ground_truth_matches.insert(0, 'record_ID', range(0, len(ground_truth_matches)))

In [ ]:
ground_truth_matches = pd.merge(ground_truth_matches,
                                ground_truth_matches,
                                on = 'unique_id')

In [ ]:
ground_truth_matches.head(5)

In [ ]:
ground_truth_matches = ground_truth_matches.query('record_ID_x < record_ID_y')
len(ground_truth_matches)

In [ ]:
ground_truth_matches = ground_truth_matches[['record_ID_x','record_ID_y']]

In [ ]:
bmatches_df = pd.DataFrame(bmatches)
bmatches_df.columns= ['record_ID_x','record_ID_y']
bmatches_df.head()

In [ ]:
# on s'assure que les couples record_ID_x et record_ID_y sont dans le bons sens (record_ID_x < record_ID_y)
bmatches_df[bmatches_df['record_ID_x'] >= bmatches_df['record_ID_y'] ]
# 0 lignes trouvées , donc c OK.

In [ ]:
diff_df = pd.merge(ground_truth_matches, bmatches_df, how='outer', indicator='Exist')
diff_df.head()

In [ ]:
btrue_positives = diff_df[diff_df.Exist=='both']
bfalse_positives = diff_df[diff_df.Exist=='right_only']
bfalse_negatives = diff_df[diff_df.Exist=='left_only']


In [ ]:
true_positives.head()

In [ ]:
# un vrai positif: c un vrai couple de restos en double qui a été détecté par notre algo sous forme de blocking method.
# en effet il vérifie le critère de name (edit_distance=0) et en plus les 2 restos se trouve dans la même ville d'atlanta.
df_restaurants[df_restaurants.record_ID.isin(['6','754'])]

In [ ]:
# les couples détectés par notre algo comme des doubles mais à tort, ce ne sont pas des doubles.
false_positives.head()

In [ ]:
df_restaurants[df_restaurants.record_ID.isin(['96','196'])]
# ce couple n'est pas dans le ground_truth car unique_id différent
# mais il est dans le bmatches_df , (jd_ng1_ng2_adr <= 0.6 and jd_ng1_ng2_name <= 0.6) 
# cad les names sont proches pour la jaccard distance item based
# et les adresses sont proches pour la jaccard distance item based.
# et en plus ils se trouvent dans la même ville atlanta (blocking method)

In [ ]:
# les couples de restos en double mais qui ne sont pas détectés par notre algo comme des doubles.
false_negatives.head()

In [ ]:
df_restaurants[df_restaurants.record_ID.isin(['2','753'])]
# ce couple est dans le ground_truth car même unique_id 
# mais il n'est pas dans le matches_df, malgré qu' ils ont le même name et la  même adresse (dans l'algo les détecte bien)
# mais le Blocking method ne permet pas à l'algo de les matcher car ils sont considérés ayant des villes différentes :
# 'new york' et 'new york city'  , à cause d'une mauvaise saisie de la ville.

In [ ]:
len(bfalse_negatives) # y a beaucoup de false_ngatives par rapport à l'algo dans Blocking method (on avait 13)

In [ ]:
# false negative
df_restaurants[df_restaurants.record_ID.isin(['26','756'])]
# du au blocking method : new yor et new york city

In [ ]:
# false negative
df_restaurants[df_restaurants.record_ID.isin(['32','759'])]
# dû aux matching imprécis de l'algo 

In [ ]:
# false negative
df_restaurants[df_restaurants.record_ID.isin(['36','760'])]
# du au blocking method : new yor et new york city

In [ ]:
print(len(ground_truth_matches))
print(len(bmatches_df))
print(len(btrue_positives) , 'true_positives')
print(len(bfalse_positives) ,'false_positives')
print(len(bfalse_negatives)  , 'false_negatives')

# len(true_positives)  +  len(false_negatives) = len(ground_truth_matches)

# len(matches_df)) - len(false_positif) + len(false_negatives)     = ground_truth_matches

In [ ]:
bprecision = len(btrue_positives)/(len(btrue_positives)+ len(bfalse_positives))
print(bprecision)

In [ ]:
brecall = len(btrue_positives)/(len(btrue_positives)+ len(bfalse_negatives))
print(brecall)
# recall faible car y a beaucoup de false negatives
# y a des duplicates que l'algo avec Blocking method n'a pas détecté car saisie à tort dans des villes différentes
# surtout new york et new york city 

In [ ]:
bf_measure = 2*(bprecision*brecall)/(bprecision+brecall)
print(bf_measure)

### chiffres de l'algo original sans blocking method
### precision: 0.7795

### recall : 0.8839

### f_measure :0.82845